In [1]:
!pip install -q langchain pypdf sentence-transformers chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.9/262.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q langchain pypdf sentence-transformers chromadb langchain-openai faiss-gpu --upgrade --quiet  rank_bm25 > /dev/null

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 로드

In [ ]:
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 외부 데이터 로드 및 split

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("/content/CNN 기반 공동주택 도배하자 검출 딥러닝 이미지 분류모델 구축에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택 건축공사의 하자유형과 대책에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택 하자발생 원인과 방지방안에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택에서 하자담보책임과 하자보수보증책임에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택의 공정별 하자발생 및 예방대책에 관한 연구.pdf"),
    PyPDFLoader("/content/공동주택의 목질계 바닥 마감재의 하자 유형 분석 및 대책.pdf"),
    PyPDFLoader("/content/벽지반점_조사연구_내화건축자재협회_최종_20151026.pdf"),
    PyPDFLoader("/content/시대별 공동주택 설계변천에 따른 하자유형 변화 및 특징에 관한 연구.pdf"),
    PyPDFLoader("/content/신축공동주택 실내마감공사 하자 원인분석 및 예방방안.pdf"),

]
docs = []
for loader in loaders:
    docs.extend(loader.load_and_split())

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(docs)

#Retriever Ensemble

In [ ]:
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2



embedding = ko_embedding
faiss_vectorstore = FAISS.from_documents(texts, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

#Retrieval, 키워드 추출

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ''

from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

openai = ChatOpenAI(model_name="gpt-4-1106-preview", temperature = 0)

qa = RetrievalQA.from_chain_type(llm = openai,
                                 chain_type = "stuff",
                                 retriever = ensemble_retriever,
                                 return_source_documents = True)

keywords=[]
query = "논문에서 도배 하자 관련 키워드 들을 리스트 형태로 뽑아줘"
result = qa(query)
result['result'].strip()
keywords.append(result['result'])
print(result['result'])



- 탈락
- 문틀과 문의 수평불량
- 개폐 관련 하자
- 도배
- 바닥판 시공
- 하자보수의 난이도
- 설치불량 하자
- 문짝에 대한 가공 최소화
- 골조의 수직, 수평 품질
- 자재 가공
- 랩핑을 이용한 마감 개선
- 열림.닫힘 관련 하자 예방
- 설치시 수직, 수평 품질 관리


In [ ]:
kewords=keywords[0].replace("-","").split('\n')

In [ ]:
keywords=keywords[:2]

# 키워드로 질문과 답변 생성


In [ ]:
def making_query(keywords):
    questions = []  # 질문을 저장할 리스트
    for keyword in keywords:
        query = "{}에 대한 질문을 가능한 많이 만들어줘".format(keyword)
        result = qa(query)
        questions.append(result['result'])  # 생성된 질문을 리스트에 추가
    return questions  # 모든 질문이 담긴 리스트 반환


questions_list = making_query(keywords)
for question in questions_list:
    print(question)


1. 탈락이란 무엇을 의미하며, 어떤 상황에서 발생하는가?
2. 문틀과 문의 수평불량이 발생하는 원인은 무엇인가?
3. 개폐 관련 하자를 방지하기 위한 주요 조치는 무엇인가?
4. 도배 작업이 하자보수에 어떤 영향을 미치는가?
5. 바닥판 시공 시 하자가 발생하는 경우, 어떤 점을 주의해야 하는가?
6. 하자보수의 난이도를 결정하는 요소는 무엇인가?
7. 설치불량 하자를 미연에 방지하기 위한 방법은 무엇인가?
8. 문짝에 대한 가공을 최소화하는 것이 중요한 이유는 무엇인가?
9. 골조의 수직, 수평 품질을 확보하기 위한 방법은 무엇인가?
10. 자재를 가공할 때 발생할 수 있는 문제점은 무엇이며, 이를 어떻게 개선할 수 있는가?
11. 랩핑을 이용한 마감이 개선되어야 하는 이유는 무엇인가?
12. 열림과 닫힘 관련 하자를 예방하기 위해 설치 시 어떤 점을 철저히 관리해야 하는가?
13. 설치 시 수직, 수평 품질 관리를 위해 어떤 검사나 절차를 따라야 하는가?
14. 문틀과 문짝 설치 시 수평을 맞추기 위한 가장 효과적인 도구나 기법은 무엇인가?
15. 설치 중 발생할 수 있는 수평불량을 식별하기 위한 가장 좋은 방법은 무엇인가?


In [ ]:
questions_list=questions_list[0].split('\n')

In [ ]:
questions_list

['1. 탈락이란 무엇을 의미하며, 어떤 상황에서 발생하는가?',
 '2. 문틀과 문의 수평불량이 발생하는 원인은 무엇인가?',
 '3. 개폐 관련 하자를 방지하기 위한 주요 조치는 무엇인가?',
 '4. 도배 작업이 하자보수에 어떤 영향을 미치는가?',
 '5. 바닥판 시공 시 하자가 발생하는 경우, 어떤 점을 주의해야 하는가?',
 '6. 하자보수의 난이도를 결정하는 요소는 무엇인가?',
 '7. 설치불량 하자를 미연에 방지하기 위한 방법은 무엇인가?',
 '8. 문짝에 대한 가공을 최소화하는 것이 중요한 이유는 무엇인가?',
 '9. 골조의 수직, 수평 품질을 확보하기 위한 방법은 무엇인가?',
 '10. 자재를 가공할 때 발생할 수 있는 문제점은 무엇이며, 이를 어떻게 개선할 수 있는가?',
 '11. 랩핑을 이용한 마감이 개선되어야 하는 이유는 무엇인가?',
 '12. 열림과 닫힘 관련 하자를 예방하기 위해 설치 시 어떤 점을 철저히 관리해야 하는가?',
 '13. 설치 시 수직, 수평 품질 관리를 위해 어떤 검사나 절차를 따라야 하는가?',
 '14. 문틀과 문짝 설치 시 수평을 맞추기 위한 가장 효과적인 도구나 기법은 무엇인가?',
 '15. 설치 중 발생할 수 있는 수평불량을 식별하기 위한 가장 좋은 방법은 무엇인가?']

In [ ]:
def making_answers(questions_list):
    answers= []  # 질문을 저장할 리스트
    for question in questions_list:
        query = "{}에 대해 답변해줘".format(question)
        result = qa(query)
        answers.append(result['result'])  # 생성된 질문을 리스트에 추가
    return answers  # 모든 질문이 담긴 리스트 반환

answer_list = making_answers(questions_list)
for answer in answer_list:
    print(answer)


탈락이란 일반적으로 어떤 것이 붙어 있던 위치에서 떨어져 나가는 것을 의미합니다. 건축이나 공동주택의 하자와 관련하여 탈락은 건축재료나 구조요소가 제자리에서 벗어나거나 떨어져 나가는 현상을 말할 수 있습니다. 이는 구조적 결함, 재료의 결함, 부적절한 시공 방법, 노후화 등 다양한 원인에 의해 발생할 수 있습니다. 예를 들어, 외벽의 타일이 떨어져 나가는 경우, 이는 탈락의 한 형태로 볼 수 있습니다.
문틀과 문의 수평불량이 발생하는 원인은 다음과 같습니다:

1. 문틀의 수직수평 불량: 문틀이 수직이나 수평을 이루지 못하고 설치되었을 때, 문짝과 문틀 사이의 정렬이 맞지 않아 개폐 시 문제가 발생할 수 있습니다.

2. 문틀 고정철물 긴결 불량: 문틀을 고정하는 철물의 연결이 제대로 이루어지지 않았을 때, 문틀이 흔들리거나 변형되어 수평이 맞지 않게 될 수 있습니다.

3. 골조의 수직, 수평 품질 불량: 건물의 골조 자체가 수직, 수평을 이루지 못하면, 그 위에 설치되는 문틀도 수평을 이루기 어렵습니다.

4. 시공 후 마감재료인 랩핑 탈락: 문틀과 문짝의 마감재료인 랩핑이 제대로 부착되지 않고 탈락하면, 문틀과 문짝 사이의 간격이 일정하지 않게 되어 수평불량을 초래할 수 있습니다.

5. 불량 자재 및 제품 제작상의 오류: 제작 과정에서의 오류나 불량 자재 사용으로 인해 문틀과 문짝이 제대로 맞지 않을 수 있습니다.

6. 시공 상의 잘못: 시공 과정에서 기능공의 숙련도 부족이나 시공 방법의 오류로 인해 문틀과 문짝이 수평을 이루지 못할 수 있습니다.

이러한 원인들로 인해 문틀과 문의 수평불량이 발생할 수 있으며, 이를 방지하기 위해서는 설계 및 계획 단계에서부터 정확한 수직, 수평을 확보하고, 고품질의 자재를 사용하며, 숙련된 기능공에 의한 정밀한 시공이 필요합니다.
개폐 관련 하자를 방지하기 위한 주요 조치로는 다음과 같은 사항들이 있습니다:

1. 설계도서에 명기된 시공 방법 준수
2. 숙련된 기능공 확보
3. 충분한 공기 부여
4. 프레임 시공 

In [ ]:
answer_list

['탈락이란 일반적으로 어떤 것이 붙어 있던 위치에서 떨어져 나가는 것을 의미합니다. 건축이나 공동주택의 하자와 관련하여 탈락은 건축재료나 구조요소가 제자리에서 벗어나거나 떨어져 나가는 현상을 말할 수 있습니다. 이는 구조적 결함, 재료의 결함, 부적절한 시공 방법, 노후화 등 다양한 원인에 의해 발생할 수 있습니다. 예를 들어, 외벽의 타일이 떨어져 나가는 경우, 이는 탈락의 한 형태로 볼 수 있습니다.',
 '문틀과 문의 수평불량이 발생하는 원인은 다음과 같습니다:\n\n1. 문틀의 수직수평 불량: 문틀이 수직이나 수평을 이루지 못하고 설치되었을 때, 문짝과 문틀 사이의 정렬이 맞지 않아 개폐 시 문제가 발생할 수 있습니다.\n\n2. 문틀 고정철물 긴결 불량: 문틀을 고정하는 철물의 연결이 제대로 이루어지지 않았을 때, 문틀이 흔들리거나 변형되어 수평이 맞지 않게 될 수 있습니다.\n\n3. 골조의 수직, 수평 품질 불량: 건물의 골조 자체가 수직, 수평을 이루지 못하면, 그 위에 설치되는 문틀도 수평을 이루기 어렵습니다.\n\n4. 시공 후 마감재료인 랩핑 탈락: 문틀과 문짝의 마감재료인 랩핑이 제대로 부착되지 않고 탈락하면, 문틀과 문짝 사이의 간격이 일정하지 않게 되어 수평불량을 초래할 수 있습니다.\n\n5. 불량 자재 및 제품 제작상의 오류: 제작 과정에서의 오류나 불량 자재 사용으로 인해 문틀과 문짝이 제대로 맞지 않을 수 있습니다.\n\n6. 시공 상의 잘못: 시공 과정에서 기능공의 숙련도 부족이나 시공 방법의 오류로 인해 문틀과 문짝이 수평을 이루지 못할 수 있습니다.\n\n이러한 원인들로 인해 문틀과 문의 수평불량이 발생할 수 있으며, 이를 방지하기 위해서는 설계 및 계획 단계에서부터 정확한 수직, 수평을 확보하고, 고품질의 자재를 사용하며, 숙련된 기능공에 의한 정밀한 시공이 필요합니다.',
 '개폐 관련 하자를 방지하기 위한 주요 조치로는 다음과 같은 사항들이 있습니다:\n\n1. 설계도서에 명기된 시공 방법 준수\n2. 숙련된 기능공

In [ ]:
import pandas as pd


# 엑셀 파일로 저장하는 함수
def save_questions_answers_to_excel(question_list, answer_list, filename='questions_answers.xlsx'):
    # 데이터 프레임 생성
    df = pd.DataFrame({
        'Question': question_list,
        'Answer': answer_list
    })
    # 엑셀 파일로 저장
    df.to_excel(filename, index=False)

# 함수 실행
save_questions_answers_to_excel(question_list, answer_list)
"질문과 답변이 'questions_answers.xlsx' 파일에 저장되었습니다."

"질문과 답변이 'questions_answers.xlsx' 파일에 저장되었습니다."